In [31]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [14]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [15]:
url = "https://www.transfermarkt.es/fc-valencia/kader/verein/1049/saison_id/2023"

In [16]:
plantilla23_24 = requests.get(url, headers=headers)


In [17]:
plantilla23_24 = BeautifulSoup(plantilla23_24.text, "html.parser")

In [18]:
temp24={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [19]:
for valor in plantilla23_24.find_all("td", class_ = "rechts hauptlink"):
    temp24["Valor"].append(valor.text)

In [20]:
temp24

{'Nombre': [],
 'Posicion': [],
 'Edad': [],
 'Nacionalidad': [],
 'Valor': ['35,00 mill. €',
  '900 mil €',
  '200 mil €',
  '15,00 mill. €',
  '10,00 mill. €',
  '4,00 mill. €',
  '2,00 mill. €',
  '30,00 mill. €',
  '2,00 mill. €',
  '6,00 mill. €',
  '2,50 mill. €',
  '16,00 mill. €',
  '8,00 mill. €',
  '20,00 mill. €',
  '15,00 mill. €',
  '2,00 mill. €',
  '800 mil €',
  '12,00 mill. €',
  '8,00 mill. €',
  '3,00 mill. €',
  '2,50 mill. €',
  '250 mil €',
  '16,00 mill. €',
  '4,00 mill. €',
  '800 mil €']}

SACAR NOMBRE JUGADOR

In [21]:
nombres = plantilla23_24.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp24["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [22]:
edad = plantilla23_24.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp24["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [23]:
posiciones = plantilla23_24.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp24["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [24]:
nacionalidades = plantilla23_24.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img")
    if img:
        #print(img[0].get("title"))
        temp24["Nacionalidad"].append(img[0].get("title"))

In [25]:
temp24

{'Nombre': ['Giorgi Mamardashvili',
  'Jaume Doménech',
  'Cristian Rivero',
  'Cristhian Mosquera',
  'Yarek Gasiorowski',
  'Mouctar Diakhaby',
  'Cenk Özkacar',
  'José Gayà',
  'Jesús Vázquez',
  'Thierry Correia',
  'Dimitri Foulquier',
  'Pepelu',
  'Hugo Guillamón',
  'Javi Guerra',
  'André Almeida',
  'Selim Amallah',
  'Pablo Gozálbez',
  'Diego López',
  'Fran Pérez',
  'Peter Federico',
  'Sergi Canós',
  'Hugo González',
  'Hugo Duro',
  'Roman Yaremchuk',
  'Alberto Marí'],
 'Posicion': ['Portero',
  'Portero',
  'Portero',
  'Defensa central',
  'Defensa central',
  'Defensa central',
  'Defensa central',
  'Lateral izquierdo',
  'Lateral izquierdo',
  'Lateral derecho',
  'Lateral derecho',
  'Pivote',
  'Pivote',
  'Mediocentro',
  'Mediocentro',
  'Mediocentro ofensivo',
  'Mediocentro ofensivo',
  'Extremo izquierdo',
  'Extremo derecho',
  'Extremo derecho',
  'Extremo derecho',
  'Extremo derecho',
  'Delantero centro',
  'Delantero centro',
  'Delantero centro'],


In [74]:
temporada24_23 =  pd.DataFrame(temp24)

In [75]:
temporada24_23.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Giorgi Mamardashvili,Portero,23,Georgia,"35,00 mill. €"
1,Jaume Doménech,Portero,33,España,900 mil €
2,Cristian Rivero,Portero,26,España,200 mil €
3,Cristhian Mosquera,Defensa central,19,España,"15,00 mill. €"
4,Yarek Gasiorowski,Defensa central,19,España,"10,00 mill. €"
5,Mouctar Diakhaby,Defensa central,27,Guinea,"4,00 mill. €"
6,Cenk Özkacar,Defensa central,23,Turquía,"2,00 mill. €"
7,José Gayà,Lateral izquierdo,28,España,"30,00 mill. €"
8,Jesús Vázquez,Lateral izquierdo,21,España,"2,00 mill. €"
9,Thierry Correia,Lateral derecho,25,Portugal,"6,00 mill. €"


In [76]:
temporada24_23.Edad = temporada24_23.Edad.astype(int)

In [77]:
temporada24_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Nombre        25 non-null     object
 1   Posicion      25 non-null     object
 2   Edad          25 non-null     int32 
 3   Nacionalidad  25 non-null     object
 4   Valor         25 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.0+ KB


In [78]:
temporada24_23.Valor = temporada24_23.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_1808\2444081181.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada24_23.Valor = temporada24_23.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [85]:
temporada24_23.loc[temporada24_23.Valor > 100,"Valor"] = temporada24_23.loc[temporada24_23.Valor > 100,"Valor"]/1000


In [87]:
temporada24_23["Valor millones"] = temporada24_23.Valor 

In [89]:
temporada24_23 = temporada24_23.drop(columns=["Valor"])

In [90]:
temporada24_23

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Giorgi Mamardashvili,Portero,23,Georgia,35.00
1,Jaume Doménech,Portero,33,España,0.90
2,Cristian Rivero,Portero,26,España,0.20
3,Cristhian Mosquera,Defensa central,19,España,15.00
4,Yarek Gasiorowski,Defensa central,19,España,10.00
5,Mouctar Diakhaby,Defensa central,27,Guinea,4.00
6,Cenk Özkacar,Defensa central,23,Turquía,2.00
7,José Gayà,Lateral izquierdo,28,España,30.00
8,Jesús Vázquez,Lateral izquierdo,21,España,2.00
9,Thierry Correia,Lateral derecho,25,Portugal,6.00


In [91]:
temporada24_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Nombre          25 non-null     object 
 1   Posicion        25 non-null     object 
 2   Edad            25 non-null     int32  
 3   Nacionalidad    25 non-null     object 
 4   Valor millones  25 non-null     float64
dtypes: float64(1), int32(1), object(3)
memory usage: 1.0+ KB


In [104]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada24_23.csv"

temporada24_23.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_1808\711130018.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada24_23.csv"
